# Collaborative Filtering and Content based movie recommendation system 

In [1]:

import pandas as pd
import numpy as np
from ast import literal_eval

credits = pd.read_csv('credits.csv.zip')
keywords = pd.read_csv('keywords.csv.zip')
links = pd.read_csv('links_small.csv')
md = pd.read_csv('movies_metadata.csv.zip')
ratings = pd.read_csv('ratings_small.csv.zip')


C:\Users\katta\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [3]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [5]:
keywords.head(5)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [6]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [7]:
links.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [9]:
md.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [10]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [11]:
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [12]:

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan
    

In [13]:
#converting id into into
md['id'] = md['id'].apply(convert_int)
md[md['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#removing rows with no id
md = md.drop([19730, 29503, 35587])

# merging using md,credits,keywords over id
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [15]:
#tmdbId in links is id in md 
a = links[links['tmdbId'].notnull()]['tmdbId'].astype('int64')
smd=md[md['id'].isin(a)].copy()
smd.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords'],
      dtype='object')

In [16]:
#getting directors name

In [17]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] =smd['crew'].apply(literal_eval)
smd['keywords'] =smd['keywords'].apply(literal_eval)
smd['cast_size'] =smd['cast'].apply(lambda x: len(x))
smd['crew_size'] =smd['crew'].apply(lambda x: len(x))

In [18]:
def getdir(x):
    for i in x:
        if i['department']=='Directing':
            return i['name']
    return np.nan

In [19]:
smd['director'] = smd['crew'].apply(getdir)

In [20]:
smd.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,cast,crew,keywords,cast_size,crew_size,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",13,106,John Lasseter


In [21]:
#getting genres in a list
smd['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [22]:
smd['genres']=smd['genres'].apply(literal_eval)
smd['genres'][0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [23]:
def getgenres(x):
    a=[]
    for i in x:
        a.append(i['name'])
    return a

In [24]:

smd['genres'] = smd['genres'].apply(getgenres)

In [25]:
smd['genres'][0]

['Animation', 'Comedy', 'Family']

In [26]:
#getting keywords in a list

In [27]:
smd['keywords'][0]

[{'id': 931, 'name': 'jealousy'},
 {'id': 4290, 'name': 'toy'},
 {'id': 5202, 'name': 'boy'},
 {'id': 6054, 'name': 'friendship'},
 {'id': 9713, 'name': 'friends'},
 {'id': 9823, 'name': 'rivalry'},
 {'id': 165503, 'name': 'boy next door'},
 {'id': 170722, 'name': 'new toy'},
 {'id': 187065, 'name': 'toy comes to life'}]

In [28]:
def getkeywords(x):
    a=[]
    for i in x:
        a.append(i['name'])
        
    return a       

In [29]:
smd['keywords']=smd['keywords'].apply(getkeywords)

In [30]:
smd['keywords'][0]

['jealousy',
 'toy',
 'boy',
 'friendship',
 'friends',
 'rivalry',
 'boy next door',
 'new toy',
 'toy comes to life']

In [31]:
#getting first 3 cast members names in a list

In [32]:
smd['cast'][0][1]

{'cast_id': 15,
 'character': 'Buzz Lightyear (voice)',
 'credit_id': '52fe4284c3a36847f8024f99',
 'gender': 2,
 'id': 12898,
 'name': 'Tim Allen',
 'order': 1,
 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}

In [33]:
def getcast(x):
    a=[]
    for i in x:
        if(len(a)<=2):
            a.append(i['name'])
    return a

In [34]:
smd['cast']=smd['cast'].apply(getcast)
smd['cast'][0]

['Tom Hanks', 'Tim Allen', 'Don Rickles']

In [35]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])    #giving director more importance

In [36]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [37]:
smd['all'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['all'] = smd['all'].apply(lambda x: ' '.join(x))

In [38]:
smd['all'][0]

'jealousi toy boy friendship friend rivalri boynextdoor newtoy toycomestolif tomhanks timallen donrickles johnlasseter johnlasseter johnlasseter Animation Comedy Family'

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=2, stop_words='english')
count_matrix = count.fit_transform(smd['all'])

#since we didnt remove words which occur only ones we set df=2 ,which will remove words which are present in less than 2 movies
#This practise is best for recommender system as we dont need to worry about a single word which only repeates in one movie

In [40]:
print(count_matrix.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [41]:
count_matrix.shape

(9219, 20144)

In [42]:
from sklearn.metrics.pairwise import  cosine_similarity
cosine_sim = cosine_similarity(count_matrix)

In [43]:
#For content based recommendation system using director,genre,cast and keywords
#cosine_sim tells how simillar two movies are based on its content 
cosine_sim[0]

array([1.        , 0.03094922, 0.03333333, ..., 0.        , 0.        ,
       0.        ])

In [44]:
cosine_sim.shape

(9219, 9219)

In [45]:
smd['title']

0                                                Toy Story
1                                                  Jumanji
2                                         Grumpier Old Men
3                                        Waiting to Exhale
4                              Father of the Bride Part II
                               ...                        
40952                       The Last Brickmaker in America
41172                                               Rustom
41225                                         Mohenjo Daro
41391                                        Shin Godzilla
41669    The Beatles: Eight Days a Week - The Touring Y...
Name: title, Length: 9219, dtype: object

In [46]:
smd = smd.reset_index()

In [47]:
smd['title']

0                                               Toy Story
1                                                 Jumanji
2                                        Grumpier Old Men
3                                       Waiting to Exhale
4                             Father of the Bride Part II
                              ...                        
9214                       The Last Brickmaker in America
9215                                               Rustom
9216                                         Mohenjo Daro
9217                                        Shin Godzilla
9218    The Beatles: Eight Days a Week - The Touring Y...
Name: title, Length: 9219, dtype: object

In [48]:
smd.columns

Index(['index', 'adult', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords', 'cast_size',
       'crew_size', 'director', 'all'],
      dtype='object')

In [49]:
#!pip install scikit-surprise
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
reader = Reader()


In [50]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [51]:
# Using collaborative filtering to estimate howmuch a user might rate a movie which they have not rated intitially by 
# finding simillar users
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.89704904, 0.89236092, 0.9041784 , 0.89204764, 0.89368561]),
 'test_mae': array([0.68958197, 0.68786826, 0.69919858, 0.6848254 , 0.68849819]),
 'fit_time': (10.905491828918457,
  10.925320863723755,
  10.92604923248291,
  11.000056982040405,
  10.900978088378906),
 'test_time': (0.29022908210754395,
  0.42366743087768555,
  0.2832937240600586,
  0.41795778274536133,
  0.2892146110534668)}

In [52]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [53]:
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])


In [54]:
id_title=links.copy()
id_title.drop('imdbId',axis=1,inplace=True)
id_title.columns=['movieId','id']
id_title=id_title.merge(smd[['title','id']], on='id').set_index('title')
id_index=id_title.set_index('id')


In [55]:
def get_recommendations(userId,title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:25]
    movie_indices = [i[0] for i in sim_scores]
    movie=smd.iloc[movie_indices][['title','id']]
    movie['est']=movie['id'].apply(lambda x: svd.predict(userId, id_index.loc[x]['movieId']).est)
    movie = movie.sort_values('est', ascending=False)
    return movie.head(10)


In [56]:

get_recommendations(1,'The Dark Knight')


,title,id,est
3381,Memento,77.0,3.289601
6623,The Prestige,1124.0,3.247250
7648,Inception,27205.0,3.191144
8031,The Dark Knight Rises,49026.0,3.119174
8613,Interstellar,157336.0,3.109921
6218,Batman Begins,272.0,2.977670
5943,Thursday,9812.0,2.967066
7561,Harry Brown,25941.0,2.892360
715,Nothing to Lose,410921.0,2.822323
4882,Sunday in the Country,163549.0,2.766329


In [57]:
get_recommendations(1, 'Avatar')

,title,id,est
522,Terminator 2: Judgment Day,280.0,3.277382
1011,The Terminator,218.0,3.110142
6179,A Trip to the Moon,775.0,2.983527
1956,Mighty Joe Young,39314.0,2.941567
8401,Star Trek Into Darkness,54138.0,2.894731
7327,Raiders of the Lost Ark - The Adaptation,62128.0,2.880632
7091,Mutant Chronicles,13256.0,2.810279
1376,Titanic,597.0,2.766827
4017,Hawk the Slayer,25628.0,2.740620
3894,Spacehunter: Adventures in the Forbidden Zone,26978.0,2.714225


In [58]:
get_recommendations(2, 'Avatar')

,title,id,est
522,Terminator 2: Judgment Day,280.0,4.170665
1011,The Terminator,218.0,3.782146
7327,Raiders of the Lost Ark - The Adaptation,62128.0,3.736497
922,The Abyss,2756.0,3.725478
6179,A Trip to the Moon,775.0,3.688729
974,Aliens,679.0,3.669678
8401,Star Trek Into Darkness,54138.0,3.659262
1956,Mighty Joe Young,39314.0,3.618666
344,True Lies,36955.0,3.613001
7091,Mutant Chronicles,13256.0,3.585402


In [59]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9219 entries, 0 to 9218
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  9219 non-null   int64  
 1   adult                  9219 non-null   object 
 2   belongs_to_collection  1688 non-null   object 
 3   budget                 9219 non-null   object 
 4   genres                 9219 non-null   object 
 5   homepage               2001 non-null   object 
 6   id                     9219 non-null   float64
 7   imdb_id                9219 non-null   object 
 8   original_language      9219 non-null   object 
 9   original_title         9219 non-null   object 
 10  overview               9207 non-null   object 
 11  popularity             9219 non-null   object 
 12  poster_path            9216 non-null   object 
 13  production_companies   9219 non-null   object 
 14  production_countries   9219 non-null   object 
 15  rele